# Session 5: Mapping Data Centers

**Goal:** Map and analyze the geographic distribution of data centers.

In this notebook, we will:
1.  Load a real dataset of data center locations.
2.  Convert raw coordinates (Lat/Lon) into spatial geometries using DuckDB.
3.  Explore patterns in Northern Virginia and across the US.

## 1. Setup and Data Loading

We use `ibis` to interact with DuckDB. DuckDB has a powerful `spatial` extension.

### Action Item 1: Setup Environment

> **Prompt your Agent:**
> "Import ibis, pandas, anymap, and geopandas. Connect to a local DuckDB instance and ensure the spatial extension is installed and loaded."

### Action Item 2: Load Data

We need to load the data center locations from a public Parquet file.

**URL:** `https://huggingface.co/datasets/cboettig/data-centers/resolve/main/data_centers.parquet`

> **Prompt your Agent:**
> "Load the parquet file from the URL provided into an Ibis table. Inspect the first few rows and check the column names."

## 2. Creating Spatial Points

Latitude and Longitude are just numbers. To do spatial analysis, we need to convert them into **Geometries** (Points).

### Action Item 3: Create Geometries

> **Prompt your Agent:**
> "Create a new column `geom` by converting the `longitude` and `latitude` columns into points using the appropriate spatial function. Make sure to cast latitude/longitude to float if necessary. Filter the data to only include data centers in the 'United States'."

## 3. Visualize with AnyMap

`anymap` allows us to visualize thousands of points interactively.

### Action Item 4: Interactive Map

> **Prompt your Agent:**
> "Convert the Ibis table to a GeoDataFrame. Use `anymap` to create an interactive map of the data center locations. Save the map as '01-data_centers.html' and display it."